**Set simulation parameters**

In [127]:
# See README for more information on simulation parameters

# Maximum simulation time, in minutes
MAXTIME = 24 * 60

# Rider info
NUM_RIDERS = 3500

# Stochastic process parameters
LAMBDA = 2.38
MU = 2.78
SIGMA = 0.619

# Station info
STATION_CAPACITY = 10
IDEALIZED = False

# Data paths
start_station_path = 'data/start_station_probs.csv'
trips_path = 'data/trip_stats.csv'

# Verbosity
VERBOSE = False

**Helper data structures and functions**

In [128]:
from dataclasses import dataclass, field
from typing import Callable

# Get and set simulation time
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(t=0)
def time():
    return time.t

def set_time(t_new=0):
    time.t = t_new
    return time()

# Store events
@dataclass(order=True)
class Event:
    t: float
    f: Callable = field(compare=False)
    i: int = field(compare=False)

class FutureEventList:
    def __init__(self):
        self.events = []

    def __iter__(self):
        return self

    def __next__(self) -> Event:
        from heapq import heappop
        if self.events:
            return heappop(self.events)
        raise StopIteration

# Schedule events
def schedule(e: Event, fev: FutureEventList):
    from heapq import heappush
    heappush(fev.events, e)

**Read data from data files**

In [129]:
import pandas as pd

# Station probabilities
all_stations = []
start_probs = {}
dest_probs = {}
if start_station_path:
    start_probs_data = pd.read_csv(start_station_path, header=0, names=['station_name', 'prob'])
    stations = start_probs_data['station_name'].to_list()
    probs = start_probs_data['prob'].to_list()
    
    for i, station in enumerate(stations):
        start_probs[station] = probs[i]

else:
    start_probs = {'A': 0.45, 'B': 0.20, 'C': 0.35}

if trips_path:
    trip_probs_data = pd.read_csv(trips_path, usecols=['start', 'end', 'count'])
    starts = trip_probs_data['start'].to_list()
    ends = trip_probs_data['end'].to_list()
    counts = trip_probs_data['count'].to_list()

    all_stations += starts
    all_stations += ends
    all_stations = set(all_stations)

    for i, start in enumerate(starts):
        end = ends[i]
        count = counts[i]
        if start in dest_probs.keys():
            dest_probs[start][end] = count
        else:
            dest_probs[start] = {end: count}

else:
    dest_probs = {'A': {'A': 0, 'B': 0, 'C': 1}, 'B': {'A': 1, 'B': 0, 'C': 0}, 'C': {'A': 0, 'B': 1, 'C': 0}}
    all_stations = ['A', 'B', 'C']
    all_stations = set(all_stations)

**Generate simulation inputs**

In [130]:
import random
import numpy as np

# Rider info
interarrival_times = np.random.exponential(1/LAMBDA, NUM_RIDERS)
arrival_times = np.cumsum(interarrival_times)
trip_durations = np.random.lognormal(MU, SIGMA, NUM_RIDERS)

# Generate starting and destination stations based on station probabilities
start_stations = random.choices(list(start_probs.keys()), weights=list(start_probs.values()), k=NUM_RIDERS)
dest_stations = [
    random.choices(list(dest_probs[start].keys()), weights=list(dest_probs[start].values()))[0]
    for start in start_stations
]

**Define and initialize the state of the simulation**

In [131]:
# Initialize state
def init_state():
    state = {
        "num_bikes": {station: STATION_CAPACITY for station in all_stations},
        "checkout_times": np.zeros(NUM_RIDERS) - 1,
        "return_times": np.zeros(NUM_RIDERS) - 1,
        "checkout_q": {},
        "return_q": {}
    }
    return state

**Define event functions**

In [132]:
def checkout_bike(state, rider_i, fev, verbose):
    station = start_stations[rider_i]

    # If bikes available, the rider takes a bike from the station
    # Schedule a return event at the destination station after a certain time
    if state["num_bikes"][station] > 0:
        state["num_bikes"][station] -= 1
        state["checkout_times"][rider_i] = time()
        travel_time = trip_durations[rider_i]
        schedule(Event(time() + travel_time, return_bike, rider_i), fev)

        if verbose:
            print(f"[t={time()}] Rider {rider_i} checks out bike at Station {station}")
            print(f"[t={time()}] Station {station} has {state["num_bikes"][station]} bikes\n")
    
    # If no bikes available, the rider joins the queue to wait for a bike
    else:
        if station in state["checkout_q"].keys():
            state["checkout_q"][station].append(rider_i)
        else:
            state["checkout_q"][station] = [rider_i]
        
        if verbose:
            print(f"[t={time()}] Rider {rider_i} joins checkout queue at Station {station}")
            print(f"[t={time()}] Station {station} checkout queue: {state["checkout_q"][station]}\n")
    
    # If there are riders waiting to return, schedule a return event for now
    if station in state["return_q"].keys() and len(state["return_q"][station]) > 0:
        returner_i = state["return_q"][station].pop(0)
        schedule(Event(time(), return_bike, returner_i), fev)

def return_bike(state, returner_i, fev, verbose):
    station = dest_stations[returner_i]

    # If space available, return the bike and exit the system
    # If simulation is idealized, space is always available
    if IDEALIZED or state["num_bikes"][station] < STATION_CAPACITY:
        state["num_bikes"][station] += 1
        state["return_times"][returner_i] = time()

        if verbose:
            print(f"[t={time()}] Rider {returner_i} returns bike at Station {station}")
            print(f"[t={time()}] Station {station} has {state["num_bikes"][station]} bikes\n")
    
    # If station is full, join the queue to wait to return the bike
    else:
        if station in state["return_q"].keys():
            state["return_q"][station].append(returner_i)
        else:
            state["return_q"][station] = [returner_i]
        
        if verbose:
            print(f"[t={time()}] Rider {returner_i} joins return queue at Station {station}")
            print(f"[t={time()}] Station {station} return queue: {state["return_q"][station]}\n")
    
    # If there are riders waiting to checkout, schedule a checkout event for now
    if station in state["checkout_q"].keys() and len(state["checkout_q"][station]) > 0:
        rider_i = state["checkout_q"][station].pop(0)
        schedule(Event(time(), checkout_bike, rider_i), fev)

**Simulate**

In [133]:
def simulate(verbose):
    state = init_state()
    event_list = FutureEventList()
    for i in range(NUM_RIDERS):
        time = arrival_times[i]
        schedule(Event(time, checkout_bike, i), event_list)

    for e in event_list:
        if set_time(e.t) > MAXTIME:
            if not IDEALIZED:
                break
        e.f(state, e.i, event_list, verbose)

    return state["checkout_times"], state["return_times"]

In [134]:
checkout_times, return_times = simulate(verbose=VERBOSE)

**Calculate simulation statistics**

In [135]:
successful_rentals = np.full_like(checkout_times, True, dtype=bool)
successful_returns = np.full_like(return_times, True, dtype=bool)

for i in range(NUM_RIDERS):
    if checkout_times[i] < 0:
        successful_rentals[i] = False
    if return_times[i] < 0:
        successful_returns[i] = False

successful_rental_rate = np.count_nonzero(successful_rentals == True) / NUM_RIDERS
successful_return_rate = np.count_nonzero(successful_returns == True) / NUM_RIDERS
wait_times = checkout_times[successful_rentals] - arrival_times[successful_rentals]
average_wait_time = np.mean(wait_times)

print(f"Successful Rental Rate: {successful_rental_rate}\nSuccessful Return Rate: {successful_return_rate}\nAverage Wait Time: {average_wait_time} minutes")

Successful Rental Rate: 0.9562857142857143
Successful Return Rate: 0.8848571428571429
Average Wait Time: 7.910665383372745 minutes
